In [1]:
import pandas as pd
import numpy as np
import os.path

from keras.preprocessing.image import Iterator
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
from keras.models import load_model

import threading
lock = threading.Lock()

from keras.callbacks import ModelCheckpoint

from BsonIterator import BSONIterator

Using TensorFlow backend.


In [4]:
METADATA_DIR = 'metadata'
DATA_DIR = '../../dataset'

train_offsets_df = pd.read_csv(os.path.join(METADATA_DIR,"train_offsets.csv"), index_col=0)
train_images_df = pd.read_csv(os.path.join(METADATA_DIR,"RandomSplit_Train_0.9_0.1.csv"), index_col=0)
val_images_df = pd.read_csv(os.path.join(METADATA_DIR,"RandomSplit_Val_0.9_0.1.csv"), index_col=0)

num_classes = 5270
num_train_images = len(train_images_df)
num_val_images = len(val_images_df)
batch_size = 64

# Create a generator for training and a generator for validation.
# Tip: use ImageDataGenerator for data augmentation and preprocessing.
assert os.path.exists(os.path.join(DATA_DIR, 'train.bson'))
train_bson_file = open(os.path.join(DATA_DIR, 'train.bson'), "rb")
train_datagen = ImageDataGenerator()
train_gen = BSONIterator(train_bson_file, train_images_df, train_offsets_df,
                         num_classes, train_datagen, lock, target_size = (224,224),
                         batch_size=batch_size, shuffle=True)

val_datagen = ImageDataGenerator()
val_gen = BSONIterator(train_bson_file, val_images_df, train_offsets_df,
                       num_classes, val_datagen, lock, target_size = (224,224),
                       batch_size=batch_size, shuffle=True)

Found 1112569 images belonging to 5270 classes.
Found 119010 images belonging to 5270 classes.


In [5]:
from keras.applications.mobilenet import MobileNet
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K

from keras.callbacks import ModelCheckpoint

# Save model Dir
MODEL_DIR = 'saved_models'

# Checkpointer 
checkpointer = ModelCheckpoint(filepath=MODEL_DIR+'/Mobilenet.{epoch:02d}-{acc:.2f}-{val_acc:.2f}_RandomSplit_Train_0.9_0.1.h5', verbose=1)

# create the base pre-trained model
base_model = MobileNet(weights='imagenet', include_top=False, input_shape=(224,224,3))

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
# and a logistic layer 
predictions = Dense(num_classes, activation='softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions, name= 'Mobilenet')

# compile the model (should be done *after* setting layers to non-trainable)
# model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.compile(optimizer="adam",
              loss="categorical_crossentropy",
              metrics=["accuracy"])

num_epochs = 5

# train the model on the new data for a few epochs
model.fit_generator(train_gen,
                    steps_per_epoch = num_train_images // batch_size,
                    epochs = num_epochs,
                    validation_data = val_gen,
                    validation_steps = num_val_images // batch_size,
                    workers = 4, callbacks=[checkpointer])


Epoch 1/5
17383/17383 [==============================] - ETA: 0s - loss: 4.0593 - acc: 0.3340 - val_loss: 3.4256 - val_acc: 0.3940
Epoch 2/5
17383/17383 [==============================] - ETA: 0s - loss: 3.2278 - acc: 0.4264 - val_loss: 3.1450 - val_acc: 0.4366
Epoch 3/5
17383/17383 [==============================] - ETA: 0s - loss: 2.9216 - acc: 0.4616 - val_loss: 2.9400 - val_acc: 0.4683
Epoch 4/5
17383/17383 [==============================] - ETA: 0s - loss: 2.7191 - acc: 0.4855 - val_loss: 2.8260 - val_acc: 0.4884
Epoch 5/5
17383/17383 [==============================] - ETA: 0s - loss: 2.5635 - acc: 0.5037 - val_loss: 2.7998 - val_acc: 0.4921


In [7]:
num_epochs = 5


# train the model on the new data for a few epochs
model.fit_generator(train_gen,
                    steps_per_epoch = num_train_images // batch_size,
                    epochs = num_epochs,
                    validation_data = val_gen,
                    validation_steps = num_val_images // batch_size,
                    workers = 4, callbacks=[checkpointer])

Epoch 1/5
17383/17383 [==============================] - ETA: 0s - loss: 2.2296 - acc: 0.5453 - val_loss: 2.6796 - val_acc: 0.5179
Epoch 2/5
17383/17383 [==============================] - ETA: 0s - loss: 2.1484 - acc: 0.5555 - val_loss: 2.6707 - val_acc: 0.5185
Epoch 3/5
17383/17383 [==============================] - ETA: 0s - loss: 2.0727 - acc: 0.5659 - val_loss: 2.6710 - val_acc: 0.5209
Epoch 4/5
 1366/17383 [=>............................] - ETA: 9008s - loss: 1.9127 - acc: 0.5865

KeyboardInterrupt: 

In [8]:
model.save(MODEL_DIR+'/Mobilenet.Epochs10_Acc0.58_Val0.52_RandomSplit_Train_0.9_0.1.h5')